<a href="https://colab.research.google.com/github/tomoyahiroe/transformers-playground/blob/main/how_tokenizer_insert_special_token.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# transformersのインストール
!pip install transformers[ja,sentencepiece,torch]

In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM, AutoModelForQuestionAnswering
import pprint
import torch

## どのようにTokenizerは特殊トークンを挿入しているのか

### cl-tohoku/bert-base-japanese-v3の例

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("cl-tohoku/bert-base-japanese-v3")
tokenizer.special_tokens_map

- 文頭と文末に特殊トークンが挿入される例

In [ ]:
text = "今日は良い天気ですね。"
encoded_inputs = tokenizer(text, return_tensors="pt")
decoded_tokens = tokenizer.convert_ids_to_tokens(encoded_inputs["input_ids"][0])
print(decoded_tokens)

- 複数の文章をひとまとまりの文字列として入力しても、途中に`[SEP]`は挿入されない

In [ ]:
text = "今日は良い天気ですね。公園で本でも読みましょうか"
encoded_inputs = tokenizer(text, return_tensors="pt")
decoded_tokens = tokenizer.convert_ids_to_tokens(encoded_inputs["input_ids"][0])
print(decoded_tokens)

- 一文毎に分割して配列にして渡せば、`[SEP]`が入力されるが、`[CLS]`も入力される

In [ ]:
text = ["今日は良い天気ですね。","公園で本でも読みましょうか"]
encoded_inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)

# input_ids をトークンにデコードして確認します
for i, input_id_list in enumerate(encoded_inputs["input_ids"]):
    decoded_tokens = tokenizer.convert_ids_to_tokens(input_id_list)
    print(f"original text: {text[i]}")
    print(f"model input(decoded): {decoded_tokens}")
    print(f"attention mask: {encoded_inputs['attention_mask'][i].tolist()}")
    print("-" * 20)

- tokenizerの引数に二つ入力することで、`[SEP]` のみを挿入できる

In [ ]:
text1 = "今日は良い天気ですね。"
text2 = "公園で本でも読みましょうか"

In [ ]:
encoded_inputs = tokenizer(text1, text2, return_tensors="pt")
decoded_tokens = tokenizer.convert_ids_to_tokens(encoded_inputs["input_ids"][0])
print(decoded_tokens)

## abeja/gpt2-large-japaneseの例

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("abeja/gpt2-large-japanese")
tokenizer.special_tokens_map

In [ ]:
text = "日本の首都は"
encoded_inputs = tokenizer(text, return_tensors="pt")
decoded_tokens = tokenizer.convert_ids_to_tokens(encoded_inputs["input_ids"][0])
print(decoded_tokens)
pprint.pp(encoded_inputs)

In [ ]:
text = "アメリカの首都はワシントンD.C.です。日本の首都は"
encoded_inputs = tokenizer(text, return_tensors="pt")
decoded_tokens = tokenizer.convert_ids_to_tokens(encoded_inputs["input_ids"][0])
print(decoded_tokens)

In [ ]:
text = ["アメリカの首都はワシントンD.C.です。", "日本の首都は"]
encoded_inputs = tokenizer(text1, text2, return_tensors="pt")
decoded_tokens = tokenizer.convert_ids_to_tokens(encoded_inputs["input_ids"][0])
print(decoded_tokens)

In [ ]:
text1 = "アメリカの首都はワシントンD.C.です。"
text2 = "日本の首都は"
encoded_inputs = tokenizer(text1, text2, return_tensors="pt")
decoded_tokens = tokenizer.convert_ids_to_tokens(encoded_inputs["input_ids"][0])
print(decoded_tokens)

## 文章を複数入力したい２つのシチュエーションについて

1. 一つの出力を得たいが、BERTに文章を区別してもらいたい

2. 一つ一つの文章毎に、BERTに特定のタスクを行ってもらいたい

In [ ]:
model = AutoModelForCausalLM.from_pretrained("abeja/gpt2-large-japanese")

### 一つの出力を得たい

#### tokenizerの引数に２つの文字列を渡すパターン

In [ ]:
context = "アメリカの首都はワシントンD.C.です。"
question = "そして、日本の首都は"

In [ ]:
encoded_inputs = tokenizer(context, question, return_tensors="pt")
decoded_tokens = tokenizer.convert_ids_to_tokens(encoded_inputs["input_ids"][0])
print(decoded_tokens)

In [ ]:
output = model.generate(**encoded_inputs, max_length=20, pad_token_id=tokenizer.pad_token_id)

In [ ]:
decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)
print(decoded_output)

#### tokenizerに配列を渡すパターン

In [ ]:
context = "アメリカの首都はワシントンD.C.です。"
question = "そして、日本の首都は"
text = [context, question]

In [ ]:
encoded_inputs = tokenizer(text, return_tensors="pt", padding=True)
for i, input_id_list in enumerate(encoded_inputs["input_ids"]):
    decoded_tokens = tokenizer.convert_ids_to_tokens(input_id_list)
    print(f"original text: {text[i]}")
    print(f"model input(decoded): {decoded_tokens}")
    print(f"attention mask: {encoded_inputs['attention_mask'][i].tolist()}")
    print("-" * 20)

In [ ]:
output = model.generate(**encoded_inputs, max_length=20, pad_token_id=tokenizer.pad_token_id)

In [ ]:
decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)
print(decoded_output)